In [4]:
import os
import json

# Path to your JSON file
json_path = "bg_noises.json"

# Base directory where the audio files live
# If they’re in the same folder as the notebook, use "."
base_dir = ""

In [5]:
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"Loaded {len(data)} entries from {json_path}")
data[:2]  # quick peek

Loaded 10 entries from bg_noises.json


[{'id': 'BG_01',
  'file_name': 'birdsong_and_wind_through_trees_in_stockwood_park.wav',
  'path': 'final/birdsong_and_wind_through_trees_in_stockwood_park.wav',
  'title': 'Gentle Park Sounds',
  'intensity': 'quiet',
  'environment': 'outdoors',
  'description': "This audio features calming sounds of birdsong and wind rustling through trees. The overall atmosphere is peaceful and relaxing, creating a serene backdrop. It's a pleasant soundscape perfect for unwinding."},
 {'id': 'BG_02',
  'file_name': 'car_driving_ambience.wav',
  'path': 'final/car_driving_ambience.wav',
  'title': 'Car Driving Ambiance',
  'intensity': 'moderate',
  'environment': 'vehicle',
  'description': "This audio features the consistent sound of a car driving on a road. You can hear the engine hum and the low rumble of the tires on the asphalt. It's a continuous sound, creating a calming and slightly hypnotic atmosphere."}]

In [6]:
missing = []
existing = []

for item in data:
    # assuming structure: {"path": "filename.wav", "description": "..."}
    rel_path = item.get("path")
    if not rel_path:
        missing.append({"path": rel_path, "reason": "missing path key"})
        continue

    full_path = os.path.join(base_dir, rel_path)
    if os.path.exists(full_path):
        existing.append(full_path)
    else:
        missing.append({"path": rel_path, "full_path_tried": full_path})

print(f"Existing files: {len(existing)}")
print(f"Missing files: {len(missing)}")

if missing:
    print("\nMissing entries:")
    for m in missing:
        print(m)

Existing files: 10
Missing files: 0
